# Prices

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### CPI components 


https://www.bls.gov/cpi/tables/relative-importance/home.htm

In [ ]:
base = 'CUUR0000'

series1 = {'SA0': 'All items',
           'SAM': 'Medical care',
           'SAH': 'Housing',
           'SEFV': 'Food away from home',
           'SAF11': 'Food at home',
           'SAR': 'Recreation',
           'SAE1': 'Education',
           'SAT': 'Transportation',
           'SAA': 'Apparel',
           'SA0E': 'Energy',
           'SAE2': 'Communication'}

series2 = {'SEHA': 'Rent of primary residence',
           'SEHC': "Owners' equivalent rent of primary residence",
           'SAH3': 'Household furnishings and operations',
           'SETA01': 'New vehicles',
           'SETA02': 'Used cars and trucks',
           'SEMD': 'Hospital and related services',
           'SEMC': 'Professional services',
           'SEME': 'Health insurance'}

series = {**series1, **series2}
series = {base+key: value for key, value in series.items()}

url = 'https://www.bls.gov/cpi/tables/relative-importance/2021.htm'
t = pd.read_html(url,header=1, index_col=0)
wgts = t[0].dropna()['CPI-U'].to_dict()
wgt_date = pd.to_datetime('2021-12-01')

In [ ]:
# Start year and end year
years = (2017, 2022)
df = bls_api(series, years, bls_key)
df.to_csv(data_dir / 'cpi_comp_raw.csv', index_label='date')

In [ ]:
df = pd.read_csv(data_dir / 'cpi_comp_raw.csv', index_col='date')
df.index = pd.to_datetime(df.index)
dates = {'Latest': df.index[-1], 'Previous': df.index[-13]}

d = {}

for name, date in dates.items():

    updated_wgts = {srs: ((df.loc[date, srs] / df.loc[wgt_date, srs])
                     * float(wgts[srs]) / 
                     (df.loc[date, 'All items'] / 
                      df.loc[wgt_date, 'All items']))
                    for key, srs in series.items()}
    
    d[name] = {srs: (df[srs].pct_change(12).loc[date] 
                        * (updated_wgts[srs]))
                  for key, srs in series.items()}
    
results = pd.DataFrame(d).round(2).sort_values('Latest', ascending=False)
results.loc[series1.values()].drop('All items').to_csv(data_dir / 'cpi_comp.csv', index_label='name')

write_txt(text_dir / 'cpi_mo1.txt', dtxt(df.index[-1])['mon2'])
write_txt(text_dir / 'cpi_mo2.txt', dtxt(df.index[-13])['mon2'])

results

In [ ]:
ltdate = dtxt(df.index[-1])['mon1']
prdate = dtxt(df.index[-13])['mon1']

data = results.loc[series1.values()].drop('All items')
c1n = data.index[0].lower()
c1v = data.Latest.iloc[0]
c1vt = value_text(c1v, style="contribution_to", ptype="pp", digits=2, casual=True)
c1 = f'In {ltdate}, {c1n} {c1vt} overall CPI inflation'
p1v = data.Previous.iloc[0]
ct1 = compare_text(c1v, p1v, cutoffs=[0.05, 0.3, 1])
p1vt = (value_text(p1v, style='contribution_of', ptype='pp', digits=2)
        .replace('a ', f"the category's {prdate} "))

c2n = data.index[1]
c2v = data.Latest.iloc[1]
c2vt = value_text(c2v, style="contribution_to", ptype="pp", digits=2)
c2 = f'{c2n} {c2vt} inflation'
p2v = data.Previous.iloc[1]
if abs(p2v - c2v) > 0.2:
    ct2 = compare_text(c2v, p2v, cutoffs=[0.05, 0.3, 1])
else:
    ct2 = 'and'
p2vt = value_text(p2v, style='contribution', ptype='pp', digits=2, casual=True)

c5n = data.index[2]
c5v = data.Latest.iloc[2]
c5vt = value_text(c5v, style="contribution_to", ptype="pp", digits=2)
c5 = f'{c5n} {c5vt} inflation'
p5v = data.Previous.iloc[2]
if abs(p5v - c5v) > 0.2:
    ct5 = compare_text(c5v, p5v, cutoffs=[0.05, 0.3, 1])
    style = 'contribution_of'
else:
    ct5 = 'and'
    style = 'contribution'
p5vt = value_text(p5v, style=style, ptype='pp', digits=2, casual=True)

c3n = data.index[-1]
c3v = data.Latest.iloc[-1]
c3vt = value_text(c3v, style="contribution_to", ptype="pp", digits=2)
c3 = f'{c3n} {c3vt} inflation in {ltdate}'
p3v = data.Previous.iloc[-1]
ct3 = compare_text(c3v, p3v, cutoffs=[0.05, 0.3, 1])
p3vt = (value_text(p3v, style='contribution_of', ptype='pp', digits=2)
        .replace('a ', 'the ').replace('an ', 'the '))

c4n = data.index[-2]
c4v = data.Latest.iloc[-2]
c4vt = value_text(c4v, style="contribution", ptype="pp", digits=2, casual=True)
c4 = f'{c4n} {c4vt} in {ltdate}'
p4v = data.Previous.iloc[-2]
ct4 = compare_text(c4v, p4v, cutoffs=[0.05, 0.3, 1])
p4vt = (value_text(p4v, style='contribution_of', ptype='pp', digits=2, casual=True)
        .replace('a ', 'the ').replace('an ', 'the '))

text = (f'{c1}, {ct1} {p1vt}. {c2} in {ltdate} {ct2} {p2vt} in {prdate}. '+
        f'{c5}, {ct5} {p5vt} one year prior.\n\n'+
        f'{c3}, {ct3} {p3vt} during the same month one year prior. '+
        f'{c4}, {ct4} {p4vt} in {prdate}.')
write_txt(text_dir / 'cpicomp.txt', text)
print(text)

### CPI Table

In [ ]:
df = pd.read_csv(data_dir / 'cpi_comp_raw.csv', 
                 index_col='date', parse_dates=True)
ltdt = df.index[-1]

data = df.pct_change(12) * 100

tbl = pd.DataFrame()
for i in [-1, -2, -3, -4, -13, -25]:
    tbl[df.index[i]] = data.iloc[i]
    
dt = '2020-02-01'
cdt = pd.to_datetime(dt)
tbl[cdt] = data.loc[dt]

tbl = tbl[tbl.columns.sort_values(ascending=False)]
tbl = tbl.applymap('{:,.1f}'.format)
tbl.columns = [dtxt(col)['mon6'] for col in tbl.columns]

wgt_dt = dtxt(ltdt)['mon6']
wgt_col = f'Weight, {wgt_dt}'
tbl[wgt_col] = pd.Series(updated_wgts).astype('float').map('{:,.3f}'.format)
tbl.loc['All items', wgt_col] = '100.0'

order = ['All items', 'Housing', "Owners' equivalent rent of primary residence",
         'Rent of primary residence', 'Household furnishings and operations',  
         'Transportation', 'New vehicles',
         'Used cars and trucks', 'Medical care', 'Professional services',
         'Hospital and related services', 'Health insurance',
         'Food at home', 'Food away from home', 'Energy', 'Recreation',
         'Communication', 'Education', 'Apparel']
tbl = tbl.loc[order]
s2adj = {val: f'\hspace{{2mm}} {val}' for val in list(series2.values())}
s1 = {val: val for val in series1.values()}
adj = {**s2adj, **s1}
tbl.index = tbl.index.map(adj)
tbl = (tbl.rename({"\hspace{2mm} Owners' equivalent rent of primary residence": 
                   "\hspace{2mm} Owners' equivalent rent"})
          .rename({'\hspace{2mm} Household furnishings and operations':
                   '\hspace{2mm} Household furnishings \& ops.'}))
(tbl.to_csv(data_dir / 'cpi_comp.tex', sep='&', 
           line_terminator='\\\ ', quotechar=' '))
tbl

In [ ]:
ltdate = dtxt(ltdt)['mon1']

res = data.loc[[ltdt, cdt]].T.sort_values(by=[ltdt])

hc = res.loc['Housing', ltdt]
h1 = value_text(res.loc['Housing', ltdt])
hp = res.loc['Housing', cdt]
hch = compare_text(hc, hp, [0.3, 1.0, 3.0])
m1 = value_text(res.loc['Medical care', ltdt])
mpr = value_text(res.loc['Medical care', cdt], casual=True)
fah1 = value_text(res.loc['Food at home', ltdt])
fahpr = res.loc['Food at home', cdt]
tc = res.loc['Transportation', ltdt]
t1 = value_text(tc)
tp = res.loc['Transportation', cdt]
tpr = value_text(tp, style='increase_end').replace('a ', '').replace('an ', '')
tch = compare_text(tc, tp, [0.3, 1.0, 3.0])
e1 = value_text(res.loc['Energy', ltdt])
epr = value_text(res.loc['Energy', cdt], style='increase_end')

text = (f'Housing prices {h1} over the year ending {ltdate}, '+
        f'{hch} the pre-COVID rate of {hp:.1f} percent (covering the year '+
        f'ending February 2020). Medical care prices {m1}, '+
        f'these prices {mpr} over the year ending February 2020. '+
        'In contrast, prices of food consumed at home '+
        f'(groceries) {fah1} in the year ending '+
        f'{ltdate} compared to {fahpr:.1f} percent over the year '+
        'ending February 2020. \n\n'+
        f'Transportation prices {t1} over the year ending {ltdate}, '+
        f'{tch} the pre-COVID {tpr}. '+
        f'Energy prices {e1} in the latest month, '+
        f'compared to {epr} in February 2020. Energy prices '+
        'are historically more volatile than other categories. ')
write_txt(text_dir / 'cpicomp2.txt', text)
print(text)

### CPI Decomposition

In [ ]:
# cols = list(rel_wgt.keys())
# d = {c: {} for c in cols}
# data = {c: {} for c in cols}
# dates = [f'{i}-12-01' for i in range(2009, 2023, 2)]
# for s, i in itertools.product(cols, dates):
#     start, end, prev = (i, dtxt(pd.to_datetime(i) + 
#                          pd.DateOffset(years=2))['datetime'],
#                         dtxt(pd.to_datetime(i) - 
#                          pd.DateOffset(years=2))['datetime'])
#     ri = [w[1] for w in rel_wgt[s] if w[0][0] == start][0]
#     base = df.loc[start, s]
#     print(s)
#     dt = pd.to_datetime(i)
#     d10 = pd.to_datetime('2010-01-01')
#     prev_link = d[s][prev] if dt > d10 else df.loc[start, s]
#     print(prev_link)
#     val = (ri * df.loc[start:end, s]) / prev_link
#     d[s][start] = (val[-1] / val[0]) * df.loc[:end, s].iloc[-1]
#     data[s].update(val.to_dict())

In [14]:
# Relative weights for series of interest, from here: 
# https://www.bls.gov/cpi/tables/relative-importance/home.htm
rel_wgt = {'CUUR0000SAF1': [(('2009-12-01', '2011-12-01'), 13.738),
                           (('2011-12-01', '2013-12-01'), 14.308),
                           (('2013-12-01', '2015-12-01'), 13.891), 
                           (('2015-12-01', '2017-12-01'), 14.015), 
                           (('2017-12-01', '2019-12-01'), 13.384),
                           (('2019-12-01', '2021-12-01'), 13.771),
                           (('2021-12-01', '2023-12-01'), 13.370)],
           'CUUR0000SA0': [(('2009-12-01', '2011-12-01'), 100.0),
                           (('2011-12-01', '2013-12-01'), 100.0),
                           (('2013-12-01', '2015-12-01'), 100.0), 
                           (('2015-12-01', '2017-12-01'), 100.0), 
                           (('2017-12-01', '2019-12-01'), 100.0),
                           (('2019-12-01', '2021-12-01'), 100.0),
                           (('2021-12-01', '2023-12-01'), 100.0)],
           'CUUR0000SA0E': [(('2009-12-01', '2011-12-01'), 8.553),
                            (('2011-12-01', '2013-12-01'), 9.679),
                            (('2013-12-01', '2015-12-01'), 9.046), 
                            (('2015-12-01', '2017-12-01'), 6.816), 
                            (('2017-12-01', '2019-12-01'), 7.513),
                            (('2019-12-01', '2021-12-01'), 6.706),
                            (('2021-12-01', '2023-12-01'), 7.348)],
           'CUUR0000SAH1': [(('2009-12-01', '2011-12-01'), 32.289),
                            (('2011-12-01', '2013-12-01'), 31.539),
                            (('2013-12-01', '2015-12-01'), 32.029), 
                            (('2015-12-01', '2017-12-01'), 33.15), 
                            (('2017-12-01', '2019-12-01'), 32.843),
                            (('2019-12-01', '2021-12-01'), 33.158),
                            (('2021-12-01', '2023-12-01'), 32.946)],
           'CUUR0000SACL1E': [(('2009-12-01', '2011-12-01'), 21.276),
                              (('2011-12-01', '2013-12-01'), 19.852),
                              (('2013-12-01', '2015-12-01'), 19.71), 
                              (('2015-12-01', '2017-12-01'), 19.613), 
                              (('2017-12-01', '2019-12-01'), 19.849),
                              (('2019-12-01', '2021-12-01'), 20.137),
                              (('2021-12-01', '2023-12-01'), 21.699)],
           'CUUR0000SASLE': [(('2009-12-01', '2011-12-01'), 56.432),
                             (('2011-12-01', '2013-12-01'), 56.161),
                             (('2013-12-01', '2015-12-01'), 57.353), 
                             (('2015-12-01', '2017-12-01'), 59.556), 
                             (('2017-12-01', '2019-12-01'), 59.254),
                             (('2019-12-01', '2021-12-01'), 59.387),
                             (('2021-12-01', '2023-12-01'), 57.583)]}
series = {key: key for key, value in rel_wgt.items()}

In [ ]:
# Start year and end year
years = (2008, 2022)
df = bls_api(series, years, bls_key)
df.to_csv(data_dir / 'cpi_decomp_raw.csv', index_label='date')

In [15]:
df = pd.read_csv(data_dir / 'cpi_decomp_raw.csv', index_col='date', 
                 parse_dates=True)

# Dictionary combining all the info for each series
d = {i: {'name': i,
         'values': df[i],
         'rel_wgt': rel_wgt[i]} for i in list(rel_wgt.keys())}

# Adjust for changes to relative importance
df1, df2, df3, df4, df5, df6, df7 = (pd.DataFrame(), pd.DataFrame(), 
                                     pd.DataFrame(), pd.DataFrame(), 
                                     pd.DataFrame(), pd.DataFrame(), 
                                     pd.DataFrame())
for i, v in d.items():
    start, end = v['rel_wgt'][0][0][0], v['rel_wgt'][0][0][1]
    rwc, rwn = v['rel_wgt'][0][1], v['rel_wgt'][1][1]
    df1.at[start: end, i] = (v['values'].loc[start: end])
    df1[i] = (df1[i].diff().cumsum() / df1.loc[start, i] + 1)
    df1.at[start, i] = 1.0
    df1[i] = (df1[i] * rwc)
    link = (df1.loc[end, i] / rwn)
    
    # Next set of dates
    start, end = v['rel_wgt'][1][0][0], v['rel_wgt'][1][0][1]
    rwc, rwn = v['rel_wgt'][1][1], v['rel_wgt'][2][1]
    df2[i] = (v['values'].loc[start: end])
    df2[i] = df2[i].diff().cumsum() / df2.loc[start, i] + 1
    df2.at[start, i] = 1.0
    df2[i] = (df2[i] * rwc) * link
    link = (df2.loc[end, i] / rwn)
    
    # Next set of dates
    start, end = v['rel_wgt'][2][0][0], v['rel_wgt'][2][0][1]
    rwc, rwn = v['rel_wgt'][2][1], v['rel_wgt'][3][1]
    df3[i] = (v['values'].loc[start: end])
    df3[i] = df3[i].diff().cumsum() / df3.loc[start, i] + 1
    df3.at[start, i] = 1.0
    df3[i] = (df3[i] * rwc) * link
    link = (df3.loc[end, i] / rwn)
    
    # Next set of dates
    start, end = v['rel_wgt'][3][0][0], v['rel_wgt'][3][0][1]
    rwc, rwn = v['rel_wgt'][3][1], v['rel_wgt'][4][1]
    df4[i] = (v['values'].loc[start: end])
    df4[i] = df4[i].diff().cumsum() / df4.loc[start, i] + 1
    df4.at[start, i] = 1.0
    df4[i] = (df4[i] * rwc) * link
    link = (df4.loc[end, i] / rwn)

    # Next set of dates
    start, end = v['rel_wgt'][4][0][0], v['rel_wgt'][4][0][1]
    rwc, rwn = v['rel_wgt'][4][1], v['rel_wgt'][5][1]
    df5[i] = (v['values'].loc[start: end])
    df5[i] = df5[i].diff().cumsum() / df5.loc[start, i] + 1
    df5.at[start, i] = 1.0
    df5[i] = (df5[i] * rwc) * link
    link = (df5.loc[end, i] / rwn)    
    
    # Next set of dates
    start, end = v['rel_wgt'][5][0][0], v['rel_wgt'][5][0][1]
    rwc, rwn = v['rel_wgt'][5][1], v['rel_wgt'][6][1]
    df6[i] = (v['values'].loc[start: end])
    df6[i] = df6[i].diff().cumsum() / df6.loc[start, i] + 1
    df6.at[start, i] = 1.0
    df6[i] = (df6[i] * rwc) * link
    link = (df6.loc[end, i] / rwn)   
    
    # Next set of dates
    start, end = v['rel_wgt'][6][0][0], v['rel_wgt'][6][0][1]
    rwc = v['rel_wgt'][6][1]
    df7[i] = (v['values'].loc[start: end])
    df7[i] = df7[i].diff().cumsum() / df7.loc[start, i] + 1
    df7.at[start, i] = 1.0
    df7[i] = (df7[i] * rwc) * link

In [33]:
# Calculate exactly what I want to show
res = pd.concat([df1, df2, df3, df4, df5, df6, df7])  # Combine the various relative importance dfs
res = res[~res.index.duplicated(keep='first')] # Drop duplicate pivot year data
final = ((res.diff(12).divide(res['CUUR0000SA0'].diff(12), axis=0))
         .multiply(res['CUUR0000SA0'].pct_change(12) * 100, axis=0))
# Core services is services less food, energy, and shelter
final['core_services'] = final['CUUR0000SASLE'] - final['CUUR0000SAH1']
# Combine food and energy
final['food_energy'] = final['CUUR0000SAF1'] + final['CUUR0000SA0E']
final = final.dropna().round(2)
d2 = final[['CUUR0000SACL1E', 'core_services', 'CUUR0000SAH1', 'food_energy']].loc['2011-01-01':]
# Rename columns
col_names = ['core_goods', 'core_services', 'shelter', 'food_energy']
d2.columns = col_names
d2['total'] = final['CUUR0000SA0'].loc['2011-01-01':]

d2.to_csv(data_dir / 'cpi_decomp.csv', index_label='date')

ltdate = dtxt(d2.index[-1])['mon1']
prdate = dtxt(d2.index[-13])['mon1']
cg = cont_subt(d2.core_goods.iloc[-1])
cs = cont_subt(d2.core_services.iloc[-1], style='end')
sh = cont_subt(d2.shelter.iloc[-1], style='end')
fe = cont_subt(d2.food_energy.iloc[-1], style='end')
tot = d2.total.iloc[-1]
cgpr = cont_subt(d2.core_goods.iloc[-13], style='end')
cspr = cont_subt(d2.core_services.iloc[-13], style='end')
shpr = cont_subt(d2.shelter.iloc[-13], style='end')
fepr = cont_subt(d2.food_energy.iloc[-13], style='end')
totpr = d2.total.iloc[-13]
text = (f'As of {ltdate}, core goods {cg} the overall non-seasonally-'+
        f'adjusted CPI inflation rate of {tot:.2f} percent, '+
        f'while core services excluding shelter {cs}. Shelter {sh}, '+
        f'and food and energy {fe}. One year prior, in {prdate}, '+
        f'the corresponding CPI inflation rate was {totpr:.2f} percent; '
        f'core goods {cgpr}, core services excluding shelter {cspr}, '+
        f'shelter {shpr}, and food and energy {fepr}.')
write_txt(text_dir / 'cpi_decomp.txt', text)
print(text)

As of January 2022, core goods contributed 2.08 percentage points to the overall non-seasonally-adjusted CPI inflation rate of 7.48 percent, while core services excluding shelter contributed 0.95 percentage point. Shelter contributed 1.54 percentage points, and food and energy contributed 2.91 percentage points. One year prior, in January 2021, the corresponding CPI inflation rate was 1.40 percent; core goods contributed 0.30 percentage point, core services excluding shelter contributed 0.23 percentage point, shelter contributed 0.57 percentage point, and food and energy contributed 0.25 percentage point.


### CPI chart

In [ ]:
# Series stored as a dictionary
series = {'CUUR0000SA0': 'ALL', 
          'CUUR0000SA0L1E': 'CORE',
          'CUSR0000SA0': 'ALL_S',
          'CUSR0000SA0L1E': 'CORE_S'}

# Start year and end year
years = (1988, 2022)
df = bls_api(series, years, bls_key)
df.to_csv(data_dir / 'cpi_index.csv', index_label='date')
(df.pct_change(12).dropna() * 100).to_csv(data_dir / 'cpi.csv', index_label='date')

In [ ]:
s = pd.read_csv(data_dir / 'cpi.csv', index_col='date', parse_dates=True)
node_color = 'blue!60!cyan'
node = end_node(s.ALL, node_color, offset=-0.2, percent=True,
                date='m', full_year=True)
write_txt(text_dir / 'cpi_node.txt', node)

date = dtxt(s.index[-1])['mon1']
allitems = value_text(s['ALL'].iloc[-1])
core = value_text(s['CORE'].iloc[-1])
text = ('\href{https://www.bls.gov/cpi/}{Consumer prices} '+
        f'{allitems} over the year ending {date} '+
        f'(see {{\color{{{node_color}}}\\textbf{{---}}}}), according '+
        'to the Consumer Price Index for all urban consumers (CPI-U). '+
        'The core CPI, which does not include the more-volatile '+
        f'food and energy prices, {core} over the same one-year period '+
        '(see {\color{gray}\\textbf{---}}).')
write_txt(text_dir / 'cpi_main.txt', text)
print(text)

In [ ]:
# Line for AHE chart
s = pd.read_csv(data_dir / 'cpi.csv', index_col='date', parse_dates=True)
cpival = s['ALL_S'].iloc[-1].round(3)
cpi_txt = f'{cpival:.1f} percent'
text = ('\\addplot[dashed, ultra thick, red, sharp plot, update limits=false] '+
        f'coordinates {{({cpival},-12.5) ({cpival}, 0.5)}} node[right] at '+
        f'(axis cs:{cpival},-12.5) {{\\textbf{{CPI}} ({cpi_txt})}};')
write_txt(text_dir / 'cpi_lt_line.txt', text)

### Monthly CPI Inflation

In [ ]:
s = pd.read_csv(data_dir / 'cpi_index.csv', index_col='date', 
                parse_dates=True)
data = ((np.log(s) - np.log(s.shift(1))) * 12) * 100
data['label'] = [dt.strftime('%b\\\`%y') if dt.month == 1 
                 else dt.strftime('%b') for dt in data.index]
data.iloc[-19:].to_csv(data_dir / 'cpi_monthly.csv', 
                         index_label='date')
ltdate = dtxt(data.index[-1])['mon1']
prdate = dtxt(data.index[-2])['mon1']
ltval = data.ALL_S.iloc[-1]
prval = data.ALL_S.iloc[-2]
text = (f'In {ltdate}, the annualized one-month change '+
        f'in the consumer price index was {ltval:.1f} '+
        'percent (see\cbox{blue}), following '+
        f'{prval:.1f} percent in {prdate}. ')
write_txt(text_dir / 'cpi_monthly.txt', text)
print(text)

### PPI

In [ ]:
df = bls_api({'WPU00000000': 'PPIACO'}, (1988, 2022), bls_key)
df.to_csv(data_dir / 'ppi_index.csv', index_label='date')

In [ ]:
df = pd.read_csv(data_dir / 'ppi_index.csv', index_col='date', 
                 parse_dates=True)
ppi = (df['PPIACO'].pct_change(12) * 100).dropna()
ppi.to_csv(data_dir / 'ppi.csv', index_label='date')

node_color = 'green!80!blue'
node = end_node(ppi, node_color, date='m', percent=True, 
                full_year=True, offset=-0.1)
write_txt(text_dir / 'ppi_node.txt', node)

ch = value_text(ppi.iloc[-1])
prval = ppi.iloc[-13]
yr3val = ppi.rolling(36).mean().iloc[-1]
compare = compare_text(ppi.iloc[-1], prval, [1.0, 3.0, 5.0])
date = dtxt(ppi.index[-1])['mon1']
date2 = dtxt(ppi.index[-13])['mon1']

text = ('The Bureau of Labor Statistics \\href{https://www.bls.gov/ppi/}'+
        '{report} prices producers receive for the various goods '+
        'and services they produce. The producer price index for all '+
        f'commodities (see {{\color{{{node_color}}}\\textbf{{---}}}}) '+
        f'{ch} over the year ending {date}, {compare} the 12-month '+
        f'growth rate of {prval:.1f} percent in {date2}. Over the '+
        f'past three years, producer prices increased by {yr3val:.1f} '+
        'percent per year, on average.')
write_txt(text_dir / 'ppi_main.txt', text)
print(text)

### Import/Export Price Index

In [ ]:
# Series stored as a dictionary
series = {'EIUIR': 'Imports', 
          'EIUIQ': 'Exports',
          'EIUIREXFUELS': 'ImpExFuels',
          'EIUIR10': 'ImpFuels',
          'EIUIQEXAG': 'ExpExAg',
          'EIUIQAG': 'ExpAg'}

# Start year and end year
years = (1988, 2022)
df = bls_api(series, years, bls_key)

df.to_csv(data_dir / 'mxpi_main.csv', index_label='date')

srs = ['Imports', 'Exports']
(df[srs].pct_change(12).dropna() * 100).to_csv(data_dir / 'mxpi.csv', 
                                               index_label='date')

In [ ]:
df = pd.read_csv(data_dir / 'mxpi_main.csv', index_col='date')

df.index = pd.to_datetime(df.index)

data = (df.pct_change(12).dropna() * 100)

ltdate = dtxt(data.index[-1])['mon1']
prdate = dtxt(data.index[-2])['mon1']
prdate2 = dtxt(data.index[-3])['mon1']
mv1 = data['Imports'].iloc[-1]
mv2 = data['Imports'].iloc[-2]
mv3 = data['Imports'].iloc[-3]
mv4 = data.loc['2017-03-01': '2020-02-01', 'Imports'].mean()
m1 = value_text(mv1, casual=True, threshold=0.1, obj='plural')
m2 = value_text(mv2, style='increase_of', threshold=0.1, obj='plural')
m3 = value_text(mv3, style='increase_of', threshold=0.1, obj='plural')
mpc = value_text(mv4, threshold=0.1, obj='plural', adj='average')
mfv1 = data['ImpExFuels'].iloc[-1]
mfv2 = data['ImpExFuels'].iloc[-2]
mfv3 = data.loc['2017-03-01': '2020-02-01', 'ImpExFuels'].mean()
mf1 = value_text(mfv1, threshold=0.1, obj='plural')
mf2 = value_text(mfv2, casual=True, threshold=0.1, obj='plural')
mfpc = value_text(mfv3, threshold=0.1, obj='plural', adj='average')
if data.index[-1].year == data.index[-2].year:
    prdate = dtxt(data.index[-2])['mon3']
if data.index[-2].year == data.index[-3].year:
    prdate2 = dtxt(data.index[-3])['mon3']
if np.sign(mv2) == np.sign(mv3):
    m3 = f'{abs(mv3):.1f} percent'
ftxt = f'{m2} in {prdate} and {m3} in {prdate2}'

xv1 = data['Exports'].iloc[-1]
xv2 = data['Exports'].iloc[-2]
xv3 = data['Exports'].iloc[-3]
xv4 = data.loc['2017-03-01': '2020-02-01', 'Exports'].mean()
x1 = value_text(xv1, casual=True, threshold=0.1, obj='plural')
x2 = value_text(xv2, style='increase_of', threshold=0.1, obj='plural')
x3 = value_text(xv3, style='increase_of', threshold=0.1, obj='plural')
x4 = value_text(xv4, style='increase_of', threshold=0.1, obj='plural')
if np.sign(xv1) == np.sign(xv2):
        x2 = f'{abs(xv2):.1f} percent'
if np.sign(xv2) == np.sign(xv3):
        x3 = f'{abs(xv3):.1f} percent'
if np.sign(xv3) == np.sign(xv4):
        x4 = f'{abs(xv4):.1f} percent'
ftxt2 = (f'compared to {x2} in {prdate}, {x3} in {prdate2}, and {x4} '+
         'on average during the three years ending February 2020')

In [ ]:
text = ('The Bureau of Labor Statistics '+
        '\href{https://www.bls.gov/news.release/ximpim.nr0.htm}{report} '
        'changes in the prices of imports and exports. Over the year ending '+
        f'{ltdate}, US import prices {m1} '+
        '(see {\color{cyan!85!yellow}\\textbf{---}}), '+
        f'following {ftxt}. Excluding fuels, US import prices '+
        f'{mf1} in {ltdate} and {mf2} in {prdate}. Over the three years '+
        'ending February 2020, prior to the US COVID-19 pandemic, US import '+
        f'prices {mpc}. Excluding fuels, import prices {mfpc} during the '+
        'same three-year pre-COVID period.\n\n'+
        'Prices of US exports (see {\color{red!25!orange}\\textbf{---}}) '+
        f'{x1} over the year ending {ltdate}, {ftxt2}. ')
write_txt(text_dir / 'mxpi.txt', text)
print(text)

### PCE Price Index

In [12]:
df = pd.read_csv(data_dir / 'nipa20804.csv', 
                 index_col='date', parse_dates=True)
df[['DPCERG', 'DPCCRG']].to_csv(data_dir / 'pce_index.csv', 
                    index_label='date')
pce = pd.DataFrame()
pce['PCE'] = df['DPCERG'].pct_change(12).dropna() * 100.0
node_color = 'orange!80!yellow'
node = end_node(pce['PCE'], node_color, date='m', percent=True, 
                full_year=True, offset=-0.2)
write_txt(text_dir / 'pce_pi_node.txt', node)

pce['CORE'] = df['DPCCRG'].pct_change(12).dropna() * 100.0
pce.to_csv(data_dir / 'pce_pi.csv', index_label='date')

ltdate = dtxt(pce.index[-1])['mon1']
prdate = dtxt(pce.index[-2])['mon1']
pryrdate = dtxt(pce.index[-13])['mon1']
ltval = pce.PCE.iloc[-1]
prval = pce.PCE.iloc[-2]
pryrval = pce.PCE.iloc[-13]
ltcore = pce.CORE.iloc[-1]
prcore = pce.CORE.iloc[-2]
pryrcore = pce.CORE.iloc[-13]

text = (f'As of {ltdate}, PCE inflation, measured as the one-year '+
        f'percent change in the overall index, is {ltval:.1f} percent '+
        f'(see {{\color{{{node_color}}}\\textbf{{---}}}}), '+
        f'compared to {prval:.1f} percent in {prdate}, and {pryrval:.1f} '+
        f'percent in {pryrdate}. Core PCE inflation, which excludes food '+
        f'and energy, was {ltcore:.1f} percent in {ltdate} '+
        f'(see {{\color{{blue!60!black}}\\textbf{{---}}}}), {prcore:.1f} '+
        f'percent in {prdate}, and {pryrcore:.1f} percent in {pryrdate}.')
write_txt(text_dir / 'pce_inf_basic.txt', text)
print(text)

As of January 2022, PCE inflation, measured as the one-year percent change in the overall index, is 6.1 percent (see {\color{orange!80!yellow}\textbf{---}}), compared to 5.8 percent in December 2021, and 1.4 percent in January 2021. Core PCE inflation, which excludes food and energy, was 5.2 percent in January 2022 (see {\color{blue!60!black}\textbf{---}}), 4.9 percent in December 2021, and 1.5 percent in January 2021.


### Trimmed mean PCE

In [3]:
# Trimmed-mean PCE from Dallas Fed
url = 'https://www.dallasfed.org/research/~/media/documents/research/pce/pcehist.xls'
tmpce = (pd.read_excel(url, index_col=0, header=3, parse_dates=True)
           .loc['1988':].dropna(axis=1))
tmpce.to_csv(data_dir / 'pce_tm.csv', index_label='date')

In [4]:
df = pd.read_csv(data_dir / 'pce_tm.csv', index_col='date', 
                 parse_dates=True).loc['1989':, '12-month']
pce = pd.read_csv(data_dir / 'pce_pi.csv', index_col='date', 
                  parse_dates=True).loc['1989':, 'PCE']
df.to_csv(data_dir / 'pce_tm12.csv', index_label='date')
ltdate = dtxt(df.index[-1])['mon1']
ltval = df.iloc[-1]
prdate = dtxt(df.index[-2])['mon1']
prval = df.iloc[-2]
ltvaltxt = value_text(ltval, threshold=0.1)
diff =  ltval - pce.loc[df.index[-1]]
difftxt = value_text(diff, 'above_below', ptype='pp')
diff2 =  prval - pce.loc[df.index[-2]]
difftxt2  = value_text(diff2, 'above_below', ptype='pp')
pcval = df.loc['2017': '2019'].mean()
diff3 = pcval - pce.loc['2017': '2019'].mean()
difftxt3  = value_text(diff3, 'above_below', ptype='pp')
color = 'violet!60!magenta'
node = end_node(df, color, date='m', percent=True, 
                full_year=True, offset=-0.1)
write_txt(text_dir / 'pce_tm_node.txt', node)
text = (f'The trimmed-mean PCE price index {ltvaltxt} over the year '+
        f'ending {ltdate} (see {{\color{{{color}}}\\textbf{{---}}}}). '+
        'By excluding top and bottom categories, the trimmed-'+
        f'mean rate was {difftxt} the all-items PCE rate. In '+
        f'{prdate}, the trimmed-mean rate was {prval:.1f} percent, '+
        f'{difftxt2} the all-items rate. From 2017--2019, the average '+
        f'trimmed-mean rate was {pcval:.1f} percent, {difftxt3} the '+
        'all-items rate.')
write_txt(text_dir / 'pce_tm_basic.txt', text)
print(text)

The trimmed-mean PCE price index increased three percent over the year ending December 2021 (see {\color{violet!60!magenta}\textbf{---}}). By excluding top and bottom categories, the trimmed-mean rate was 2.7 percentage points below the all-items PCE rate. In November 2021, the trimmed-mean rate was 2.9 percent, 2.8 percentage points below the all-items rate. From 2017--2019, the average trimmed-mean rate was 1.9 percent, 0.1 percentage point above the all-items rate.


### Prices Table

In [13]:
pce = pd.read_csv(data_dir / 'pce_index.csv', 
                  index_col='date', parse_dates=True).pct_change(12) * 100
cpi = (pd.read_csv(data_dir / 'cpi_index.csv', 
                   index_col='date', parse_dates=True)
         .rename({'ALL': 'CPI', 'CORE': 'CPI_CORE'}, axis=1)).pct_change(12) * 100
ppi = pd.read_csv(data_dir / 'ppi_index.csv', 
                  index_col='date', parse_dates=True).pct_change(12) * 100
mxpi = pd.read_csv(data_dir / 'mxpi_main.csv', 
                   index_col='date', parse_dates=True).pct_change(12) * 100
res = pce.join([cpi, ppi, mxpi], how='outer')
keep_cols = ['CPI', 'CPI_CORE', 'PPIACO', 'Imports', 
             'Exports', 'DPCERG', 'DPCCRG']
tm = pd.read_csv(data_dir / 'pce_tm.csv', 
                 index_col='date', parse_dates=True)['12-month']
srs = {'CPI': 'CPI, All Items',
       'CPI_CORE': 'CPI, ex. Food \& Energy',
       'PPIACO': 'PPI, All Commodities',
       'Imports': 'Imports Price Index',
       'Exports': 'Exports Price Index',
       'DPCERG': 'PCE, All Items',
       'DPCCRG': 'PCE, ex. Food \& Energy',
       '12-month': 'PCE, Trimmed Mean'}
res12 = (res[keep_cols].join(tm, how='outer').rename(srs, axis=1))
tbl = res12.iloc[[-1, -2, -3, -4, -13, -25]].T
tbl.columns = [dtxt(c)['mon6'] for c in tbl.columns]
tbl['`17--19 Avg.'] = res12.loc['2017': '2019'].mean()
tbl['`00-- Avg.'] = res12.loc['2000':].mean()
tbl = tbl.applymap('{:.1f}'.format).replace('nan', '--')
tbl.to_csv(data_dir / 'prices_12m.tex', sep='&', line_terminator='\\\ ', 
           quotechar=' ', float_format='%g')

In [14]:
tbl

,Jan `22,Dec `21,Nov `21,Oct `21,Jan `21,Jan `20,`17--19 Avg.,`00-- Avg.
"CPI, All Items",7.5,7.0,6.8,6.2,1.4,2.5,2.1,2.3
"CPI, ex. Food \& Energy",6.0,5.5,4.9,4.6,1.4,2.3,2.1,2.1
"PPI, All Commodities",19.3,20.3,22.7,22.4,2.8,0.1,2.6,3.0
Imports Price Index,10.8,10.2,11.8,11.0,1.0,0.5,1.6,1.9
Exports Price Index,15.1,14.8,18.1,18.3,2.5,0.4,1.6,1.8
"PCE, All Items",6.1,5.8,5.6,5.1,1.4,1.9,1.8,1.9
"PCE, ex. Food \& Energy",5.2,4.9,4.7,4.2,1.5,1.8,1.8,1.8
"PCE, Trimmed Mean",--,3.0,2.9,2.6,1.7,2.1,1.9,2.0
